### Word2vec 모델을 학습시키고 사용해 보는 것이 목표
* 나무위키 덤프 활용 - 정제 - 텍스트로 된 순수 문장 얻기
* 형태소 분석(한글 처리를 위해) - 형태소 분류 : 단어, 조사, 어미등을 다 분리가능.
* 말뭉치(Corpus) 데이터 준비
* 말뭉치를 활용해서 Word2vec 모델 학습
* Word2vec 모델 활용

In [1]:
!pip install ijson
!pip install gensim

distributed 1.22.0 requires msgpack, which is not installed.
tensorboard 1.8.0 has requirement bleach==1.5.0, but you'll have bleach 2.1.3 which is incompatible.
bleach 2.1.3 has requirement html5lib!=1.0b1,!=1.0b2,!=1.0b3,!=1.0b4,!=1.0b5,!=1.0b6,!=1.0b7,!=1.0b8,>=0.99999999pre, but you'll have html5lib 0.9999999 which is incompatible.
distributed 1.22.0 requires msgpack, which is not installed.
tensorboard 1.8.0 has requirement bleach==1.5.0, but you'll have bleach 2.1.3 which is incompatible.
bleach 2.1.3 has requirement html5lib!=1.0b1,!=1.0b2,!=1.0b3,!=1.0b4,!=1.0b5,!=1.0b6,!=1.0b7,!=1.0b8,>=0.99999999pre, but you'll have html5lib 0.9999999 which is incompatible.


* gensim은 single core용 라이브러리라 오래 걸릴 경우 spark로 하면됨.

In [2]:
input_filename = '/Users/ryanshin/Downloads/namuwiki180326/namuwiki_20180326.json'
input_filename

'/Users/ryanshin/Downloads/namuwiki180326/namuwiki_20180326.json'

# ijson 라이브러리

* 파일의 일부분만 메모리에 올릴 수 있는 라이브러리 => 파일 크기가 커도 사용가능.
* github https://github.com/isagalaev/ijson

In [3]:
import ijson
input_file = open(input_filename, 'r', encoding='utf-8')
texts = ijson.items(input_file, 'item.text')

In [4]:
texts

<generator object items at 0x104e1f150>

In [5]:
for something in zip(range(3), texts):
    print(something)

(0, '#redirect 느낌표\n')
(1, '[[파일:3444050440.jpg]]\n([[신 세계수의 미궁 2]]에서 뜬 !!아앗!!)\n{{{+1 ！！ああっと！！ }}}\n\n[[세계수의 미궁 시리즈]]에 전통으로 등장하는 대사. [[세계수의 미궁 2 제왕의 성배|2편 제왕의 성배]]부터 등장했으며, 훌륭한 [[사망 플래그]]의 예시이다.\n\n세계수의 모험가들이 탐험하는 던전인 수해의 구석구석에는 채취/벌채/채굴 포인트가 있으며, 이를 위한 채집 스킬에 투자하면 제한된 채집 기회에 보다 큰 이득을 챙길 수 있다. 그러나 분배할 수 있는 스킬 포인트는 한정된 만큼 채집 스킬에 투자하는 만큼 전투 스킬 레벨은 낮아지게 된다.\n\n 1. 채집용 캐릭터들로 이루어진 약한 파티(ex: [[레인저(세계수의 미궁 2)|레인저]] 5명)가 수해에 입장한다.\n 1. 필드 전투를 회피하면서 채집 포인트에 도착해 열심히 아이템을 캐는 중에...\n 1. \'\'\'!!아앗!!\'\'\' ~~라플레시아가 나타났다!~~\n \'\'\'이때 등장하는 것은 [[FOE(세계수의 미궁 시리즈)|FOE]]는 아닌 일단 필드 졸개지만, 훨씬 위 층에 등장하는 강력한 졸개이며 선 턴을 빼앗긴다!\'\'\'\n 1. \'\'\'[[으앙 죽음|떡잎]]\'\'\'(hage)\n\n작품마다 !!아앗!!의 세세한 모습은 다르다. 그 악랄함은 첫 등장한 작품이자 시리즈 중에서도 불친절하기로 정평이 난 2편이 절정이었는데, 그야말로 위의 !!아앗!! 시퀀스 그대로, 묻지도 따지지도 않고 채집할 때마다 일정 확률로 \'\'\'강제로\'\'\' 전투에 돌입해야 했다. 게다가 이럴 때 쓰라고 있는 레인저의 스킬 \'위험감지(중간 확률로 적의 선제공격을 무효화)\'는 작동하지 않았다!\n[[세계수의 미궁 3 성해의 내방자|3편]], [[세계수의 미궁 4 전승의 거신|4편]]에는 숨통이 트이게도 채집 중 낮은 확률로 "좋은 아이템을 얻을 수 있을 것 같지만... 주변에서 몬스터들의 기척이 느껴진다."는

# 위키 문법 제거
* 검색방법 예시 : strip wiki grammer github
* 다음 코드로 위키 문법을 제거

In [6]:
import re
# We want only plain texts, so strip wiki grammer.
# Refer this link to know more about grammar. https://namu.wiki/w/%EB%82%98%EB%AC%B4%EC%9C%84%ED%82%A4:%EB%AC%B8%EB%B2%95%20%EB%8F%84%EC%9B%80%EB%A7%90

# Use regular expression to capture some pattern

# see http://stackoverflow.com/questions/2718196/find-all-chinese-text-in-a-string-using-python-and-regex
chinese = re.compile(u'[⺀-⺙⺛-⻳⼀-⿕々〇〡-〩〸-〺〻㐀-䶵一-鿃豈-鶴侮-頻並-龎]', re.UNICODE)
japanese = re.compile(u'[\u3000-\u303f\u3040-\u309f\u30a0-\u30ff\uff00-\uff9f\u4e00-\u9faf\u3400-\u4dbf]', re.UNICODE)

# hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글과 띄어쓰기를 제외한 모든 글자
# hangul = re.compile('[^ \u3131-\u3163\uac00-\ud7a3]+')  # 위와 동일
# result = hangul.sub('', s) # 한글과 띄어쓰기를 제외한 모든 부분을 제거

def strip_wiki_literal(text):               
    text = re.sub(r"\{\{\{#\!html[^\}]*\}\}\}", '', text, flags=re.IGNORECASE|re.MULTILINE|re.DOTALL) # remove html
    text = re.sub(r"#redirect .*", '', text, flags=re.IGNORECASE) # remove redirect
    text = re.sub(r"\[\[분류:.*", '', text) # remove 분류
    text = re.sub(r"\[\[파일:.*", '', text) # remove 파일
    text = re.sub(r"\* 상위 문서 ?:.*", '', text) # remove 상위문서        
    text = re.sub(r"\[youtube\(\w+\)\]", '', text, flags=re.IGNORECASE) # remove youtube
    text = re.sub(r"\[include\(([^\]|]*)(\|[^]]*)?\]", r'\1', text, flags=re.IGNORECASE) # remove include
    text = re.sub(r"\[\[(?:[^\]|]*\|)?([^\]|]+)\]\]", r'\1', text) # remove link
    text = re.sub(r"\[\*([^\]]*)\]", '', text) # remove 각주
    text = re.sub(r"\{\{\{([^\ }|]*) ([^\}|]*)\}\}\}", r'\2', text) # remove text color/size
    text = re.sub(r"'''([^']*)'''", r'\1', text) # remove text bold
    text = re.sub(r"(~~|--)([^']*)(~~|--)", '', text) # remove strike-through
    text = re.sub(r"\|\|(.*)\|\|", '', text) # remove table
    text = re.sub(r"\n\s*\n*", '\n', text) # remove empty line                       
    text = chinese.sub('', text) # remove chinese
    text = japanese.sub('', text) # remove japanese
    return text

In [7]:
for index, text in zip(range(10), texts):
    print(strip_wiki_literal(text))

틀:다른 뜻1, other1=음악에서 사용되는 조표 ♯(샤프), rd1=조표(음악))
틀:링크시 주의, 링크=[[##] 또는 \\\\#)]
##암묵의 룰에 따라서 이 문서에는 주석이 있습니다.
[목차]
== 설명 ==
해시(hash), 번호 기호, 넘버 사인
특수문자의 일종으로 모양이 비슷하다는 이유로 국내에선 흔히 우물 정()자라고 불린다. 컴퓨터는 서양에서 만들었으므로 #의 원래 명칭은 샤프라고 생각하는 사람들이 있으나 샤프도 아니고 샵에서 유래하지도 않았다. 실제로는 서양에서 쓰이던 무게 또는 숫자를 표기하는 기호에서 유래한 특수 문자이다. 즉 가짜동족어로 인해 A라고 쓰고 B라고 읽는 문자가 된 것.
기존의 #은 주로 숫자의 넘버 사인이나 무게로서의 파운드 사인으로 쓰여 왔으나, 트위터의 확산 이후로 어떤 특정한 주제로 짧은 이야기를 할 경우 그 주제를 '#(주제)' 같은 식으로 표현하는데, 이것이 해시태그 문화로 자리잡게 되었다.
그 외 전화기에서 *과 함께 특수한 기능을 하는 문자로 흔히 입력의 마무리를 할 때 쓰인다. 흔한 예시로는 "지역 번호와 전화 번호를 입력하신 후 '우물 정' 자를 눌러주십시오." 그 외에 은행에서 비밀번호를 지정할 때도 마지막에 '우물 정' 자를 찍었다. 최근에는 거의 대부분의 은행에 확인 키가 마련되어 있다.
이 기호의 정확한 명칭은 샵이 아니지만, 지금은 해체된 혼성그룹 S\#ARP의 표기에는 이 문자가 쓰였다. C\#, F\# 등의 프로그래밍 언어 이름도 마찬가지. 한편 포스코건설의 아파트 브랜드인 더샵은 공식 사이트를 보면 브랜드에 대해 "반올림을 뜻하는 음악기호 "\#“에서 유래한것"이라고 설명되어 있는데, 이 경우 정확한 기호는 #이 아닌 ♯이 될 것이다.
보물 제1878호인 호우명그릇의 첫 부분에 #이 새겨져 있는데, 소설가 최인호는 만주 등지를 뒤집고 다니며 연구한 끝에 저 호우명 그릇 바닥의 상단에 보이는 # 문장이 광개토대왕의 문장이라고 추측했다. 물론 모양이 #와 같은 것은 우연의 일치.
위키백과에서는 

In [8]:
output_filename = '/Users/ryanshin/Downloads/namuwiki180326/namuwiki_20180326_mini.txt'

In [9]:
item_limit = 10000
minimum_length = 500
with open(output_filename, 'w', encoding='utf-8') as output_file:
    for index, text in zip(range(item_limit), texts):
        if len(text) > minimum_length:
            try:
                a_line = strip_wiki_literal(text)
                output_file.write(a_line + '\n')
            except UnicodeEncodeError as e:
                print("UnicodeEncodeError ({0}) : {1}".format(e, text))